<a href="https://colab.research.google.com/github/drtophop/CASAML-Thesis/blob/main/CAS_AML_Thesis_NguyenHop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CAS AML Thesis

## Introduction

### Scope

### Contents

### Sources

### Remarks

## Libraries and Modules

In [1]:
import numpy as np

In [2]:
import os
import  time, random
import pandas as pd

In [3]:
import torch
from sklearn.model_selection import train_test_split

In [4]:
from IPython import display
from IPython.display import clear_output
from pathlib import Path

In [5]:
import yaml

In [6]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [7]:
import glob
import io
import cv2
import json

In [8]:
import shutil
from tqdm.auto import tqdm

#### Numpy

#### Opendatasets

The opendatasets Python package, provides a convenient way to download and work with open datasets from various sources

In [9]:
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import opendatasets as od

##### Clear locals

Before starting with the Yolov5, existing contents and existing folders from a potentional former execution of this Notebook will be cleared to show proper execution, by following expressions

In [14]:
dirs_to_delete = ['car-object-detection', 'yolov5', 'test', 'tmp', 'train', 'valid', 'yolov5_train']

for dir_to_delete in dirs_to_delete:
    path_del = os.path.join(path_main, dir_to_delete)
    if os.path.exists(path_del):
        shutil.rmtree(path_del)

files_to_delete = ['data.yaml','yolov5m6.pt']

for file_to_delete in files_to_delete:
    file_del = os.path.join(path_main, file_to_delete)
    if os.path.exists(file_del):
        os.remove(file_del)

In [13]:
# Prepare the String for the detection dataset folder
path_yoloset = f'{path_main}/car-object-detection/data'

## Datasets

#### Local Folder

In [11]:
path_main = os.getcwd()

### Object Recognition

In [12]:
od.download('https://www.kaggle.com/datasets/sshikamaru/car-object-detection/download?datasetVersionNumber=2')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: drtophop
Your Kaggle Key: ··········


100%|██████████| 112M/112M [00:05<00:00, 22.0MB/s]


### Object Segmentation

## Object Recoginition with Yolov5

### Clone and Setup Yolo V5

In [15]:
# Clone yolov5 from the corresponding Git Repository to Local
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 15978, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 15978 (delta 89), reused 110 (delta 70), pack-reused 15831
Receiving objects: 100% (15978/15978), 14.54 MiB | 24.90 MiB/s, done.
Resolving deltas: 100% (10963/10963), done.


In [16]:
# Install the required Python packages specified in the requirements.txt file within the yolov5 directory
!pip install -qr yolov5/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 595.6/595.6 kB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.8 MB/s eta 0:00:00


### Initialization

Set up the Hyperparameters to run yolov5

In [17]:
PROJECT_NAME = "yolov5_train"
BASE_MODEL = "yolov5m6.pt"
TRAIN_BATCH = 32
TRAIN_EPOCHS = 20
VAL_BATCH = 64


# Doublecheck if needed Folders are ok
print(f'does path_main exist: {os.path.exists(path_main)}')
print(f'does path_yoloset exist: {os.path.exists(path_yoloset)}')

does path_main exist: True
does path_yoloset exist: False


In [18]:
img_h, img_w, num_channels = (380, 676, 3)

df = pd.read_csv(f'{path_yoloset}/train_solution_bounding_boxes (1).csv')

df.rename(columns={'image':'image_id'}, inplace=True)
df['image_id'] = df['image_id'].apply(lambda x: x.split('.')[0])
df['x_center'] = (df['xmin'] + df['xmax'])/2
df['y_center'] = (df['ymin'] + df['ymax'])/2
df['w'] = df['xmax'] - df['xmin']
df['h'] = df['ymax'] - df['ymin']
df['classes'] = 0
df['x_center'] = df['x_center']/img_w
df['w'] = df['w']/img_w
df['y_center'] = df['y_center']/img_h
df['h'] = df['h']/img_h
df.head()


FileNotFoundError: ignored

In [ ]:
index = list(set(df.image_id))
image = random.choice(index)
print("Image ID: %s"%(image))

img = cv2.imread(f'{path_yoloset}/training_images/{image}.jpg')
img.shape

In [ ]:
#adde here cose

In [ ]:
source = 'training_images'

for name, mini in tqdm(df.groupby('image_id')):
    if not os.path.exists(os.path.join(path_main, 'tmp', 'labels')):
        os.makedirs(os.path.join(path_main, 'tmp', 'labels'))

    with open(os.path.join(path_main, 'tmp', 'labels', '{}.txt'.format(name)), 'w+') as f:
        row = mini[['classes', 'x_center', 'y_center', 'w', 'h']].astype(float).values
        row = row.astype(str)
        for j in range(len(row)):
            text = ' '.join(row[j])
            f.write(text)
            f.write("\n")

    if not os.path.exists(os.path.join(path_main, 'tmp', 'images')):
        os.makedirs(os.path.join(path_main, 'tmp', 'images'))

    shutil.copy(
        os.path.join(path_yoloset, source, '{}.jpg'.format(name)),
        os.path.join(path_main, 'tmp', 'images', '{}.jpg'.format(name))

In [ ]:
IMAGES_PATH = f'{path_main}/tmp/images/'
LABELS_PATH = f'{path_main}/tmp/labels/'
os.path.exists(IMAGES_PATH)

In [ ]:
# Read labels
labels = os.listdir(LABELS_PATH)


# Split data
train, test = train_test_split(labels, test_size=0.15, shuffle=True)
valid, test = train_test_split(test, test_size=0.2)

print(f"train: {len(train)}; valid: {len(valid)}; test: {len(test)}")

In [ ]:
os.makedirs(f'{path_main}/test/images')
os.makedirs(f'{path_main}/test/labels')
os.makedirs(f'{path_main}/train/images')
os.makedirs(f'{path_main}/train/labels')
os.makedirs(f'{path_main}/valid/images')
os.makedirs(f'{path_main}/valid/labels')

In [ ]:
# Function to move files to directory
def move_files_to_dir(files, dirname):
    # Create target directories if they don't exist
    os.makedirs(f'{path_main}/{dirname}/images', exist_ok=True)
    os.makedirs(f'{path_main}/{dirname}/labels', exist_ok=True)

    for label_filename in files:
        image_filename = f"{label_filename[:-4]}.jpg"
        shutil.copy(f"{IMAGES_PATH}/{image_filename}", f"{path_main}/{dirname}/images/{image_filename}")
        shutil.copy(f"{LABELS_PATH}/{label_filename}", f"{path_main}/{dirname}/labels/{label_filename}")


# Move splits to folders
move_files_to_dir(train, "train")
move_files_to_dir(test, "test")
move_files_to_dir(valid, "valid")


In [ ]:
directories = ['train', 'test', 'valid']
subdirectories = ['images', 'labels']

for directory in directories:
    for subdirectory in subdirectories:
        path = os.path.join(path_main, directory, subdirectory)
        file_count = len(os.listdir(path))
        print(f"Number of files in {path}: {file_count}")

In [ ]:
train_path = f'{path_main}/train'
test_path = f'{path_main}/test'
valid_path  = f'{path_main}/valid'

test_path

In [ ]:
print(
    f"train: {train_path}\n"
    f"test: {test_path}\n"
    f"val: {valid_path}\n"
    f"nc: {1}\n"
    f"names: HopCar",
)

with open("data.yaml", "w") as file:
    yaml.dump({
        "train": train_path,
        "test": test_path,
        "val": valid_path,
        "nc": 1,
        "names": {0: "HopCar"}
    }, file)

In [ ]:
# Delete old results if exists
wildcard = f"{PROJECT_NAME}/feature_extraction*"
! rm -r $wildcard

In [ ]:
!python yolov5/train.py --batch $TRAIN_BATCH --epochs $TRAIN_EPOCHS --data "data.yaml" --weights $BASE_MODEL --project $PROJECT_NAME --name 'feature_extraction' --cache --freeze 12

In [ ]:
# Delete old results
wildcard = f"{PROJECT_NAME}/validation_on_test_data*"
! rm -r $wildcard

In [ ]:
WEIGHTS_BEST = f"{PROJECT_NAME}/feature_extraction/weights/best.pt"
! python yolov5/val.py --weights $WEIGHTS_BEST --batch $VAL_BATCH --data 'data.yaml' --task test --project $PROJECT_NAME --name 'validation_on_test_data' --augment

In [ ]:
# Delete old results
wildcard = f"{PROJECT_NAME}/detect_test*"
! rm -r $wildcard

In [ ]:
pics = ['F1_curve.png', 'PR_curve.png', 'PR_curve.png', 'R_curve.png']

fig, axs = plt.subplots(2, 2,figsize=(10, 10))

for i, pic in enumerate(pics):
    image_path = f'{PROJECT_NAME}/feature_extraction/{pic}'
    img = mpimg.imread(image_path)
    row = i // 2
    col = i % 2
    axs[row, col].imshow(img)
    axs[row, col].axis('off')
    axs[row, col].set_title(pic[:-4].replace("_", " ").upper())

plt.tight_layout()
plt.show()


In [ ]:
pics = ['confusion_matrix.png', 'labels.jpg', 'labels_correlogram.jpg', 'results.png']

fig, axs = plt.subplots(4, 1, figsize=(8, 24))  # Adjust figsize as desired

for i, pic in enumerate(pics):
    image_path = f'{PROJECT_NAME}/feature_extraction/{pic}'
    img = mpimg.imread(image_path)
    axs[i].imshow(img)
    axs[i].axis('off')
    axs[i].set_title(pic[:-4].replace("_", " ").upper())

plt.tight_layout()
plt.show()

In [ ]:
pics = ['val_batch0_labels.jpg', 'val_batch0_pred.jpg']

fig, axs = plt.subplots(1, 2, figsize=(20, 10))  # Adjust figsize as desired

for i, pic in enumerate(pics):
    image_path = f'{PROJECT_NAME}/feature_extraction/{pic}'
    img = mpimg.imread(image_path)
    axs[i].imshow(img)
    axs[i].axis('off')
    axs[i].set_title(pic[:-4].replace("_", " ").upper())

plt.tight_layout()
plt.show()

In [ ]:
!python yolov5/detect.py --weights $WEIGHTS_BEST --conf 0.6 --source '/content/tmp/images' --project $PROJECT_NAME --name 'detect_test' --augment --line=3 --save-txt

In [ ]:
directory = "yolov5_train/detect_test"
jpg_files = [file for file in os.listdir(directory) if file.endswith(".jpg")]
random_files = random.sample(jpg_files, 5)

for file in random_files:
    image_path = os.path.join(directory, file)
    image = mpimg.imread(image_path)
    plt.imshow(image)
    plt.axis("off")
    plt.show()

In [ ]:
# Path to the output folder for cropped images
output_folder = "cropped_images"
os.makedirs(output_folder, exist_ok=True)

# Path to the directory containing the labeled images
label_folder = "yolov5_train/detect_test/labels"

# Process each labeled image
for label_file in os.listdir(label_folder):
    image_name = label_file[:-4] + ".jpg"
    image_path = os.path.join(image_folder, image_name)
    label_path = os.path.join(label_folder, label_file)
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    # Read the label file
    with open(label_path, "r") as file:
        lines = file.readlines()

    # Process each line in the label file
    for line in lines:
        class_id, x_center, y_center, box_width, box_height = map(float, line.strip().split())

        # Calculate bounding box coordinates
        x_min = int((x_center - box_width / 2) * width)
        y_min = int((y_center - box_height / 2) * height)
        x_max = int((x_center + box_width / 2) * width)
        y_max = int((y_center + box_height / 2) * height)

        # Crop and save the image using the bounding box coordinates
        cropped_img = image[y_min:y_max, x_min:x_max]
        cropped_img_name = f'{image_name[:-4]}_{int(class_id)}.jpg'
        cropped_img_path = os.path.join(output_folder, cropped_img_name)
        cv2.imwrite(cropped_img_path, cropped_img)

In [ ]:


# Path to the output folder for cropped images
output_folder = "cropped_images"
os.makedirs(output_folder, exist_ok=True)

# Path to the directory containing the labeled images
label_folder = "yolov5_train/detect_test/labels"
image_folder = "/content/tmp/images"

# Randomly select 5 images
image_files = os.listdir(label_folder)
random_images = random.sample(image_files, 5)

# Create a figure to display the images
fig, axs = plt.subplots(5, 2, figsize=(10, 20))

# Process each randomly selected image
for i, image_file in enumerate(random_images):
    image_name = image_file[:-4] + ".jpg"
    image_path = os.path.join(image_folder, image_name)
    label_path = os.path.join(label_folder, image_file)
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    # Read the label file
    with open(label_path, "r") as file:
        lines = file.readlines()

    # Process the first bounding box in the label file
    line = lines[0]
    class_id, x_center, y_center, box_width, box_height = map(float, line.strip().split())

    # Calculate bounding box coordinates
    x_min = int((x_center - box_width / 2) * width)
    y_min = int((y_center - box_height / 2) * height)
    x_max = int((x_center + box_width / 2) * width)
    y_max = int((y_center + box_height / 2) * height)

    # Crop the image using the bounding box coordinates
    cropped_img = image[y_min:y_max, x_min:x_max]

    # Display the uncropped image
    axs[i, 0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    axs[i, 0].axis("off")
    axs[i, 0].set_title("Uncropped Image")

    # Display the cropped image
    axs[i, 1].imshow(cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB))
    axs[i, 1].axis("off")
    axs[i, 1].set_title("Cropped Image")

# Adjust the spacing between subplots
plt.tight_layout()

# Show the figure
plt.show()